In [1]:
import cv2
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split
import time
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms as TF
from torchvision.transforms import functional as F

In [2]:
from naturalistic_image_embedder.third_party.td_ken_burns.public import depth_map

In [3]:
dataset_dir = '/home/vovechkin/ads/dataset'
# dataset_dir = '/home/vovechkin/generator_data/Hday2night'
image_dir = os.path.join(dataset_dir, 'composite_images')
masks_dir = os.path.join(dataset_dir, 'masks')

depth_dir = os.path.join(dataset_dir, 'depth_images')
if not os.path.exists(depth_dir):
    os.mkdir(depth_dir)

In [4]:
PROD = True

all_images = [os.path.join(image_dir, image_name) for image_name in os.listdir(image_dir)]

if not PROD:
    all_images = all_images[:10]
    sample_image_path = all_images[0]
    sample_img = cv2.imread(sample_image_path, cv2.IMREAD_COLOR)
    
    depth_image = depth_map(sample_img)
    fig = plt.figure(figsize=(15, 15))
    plt.imshow(depth_image[0])
    plt.show()

In [5]:
size = len(all_images)
image_size = (1024, 1024)

for i, image_path in enumerate(all_images):
    try:
        name, _ = os.path.basename(image_path).split('.')
        dst_name = os.path.join(depth_dir, name) + '.png'

        image = cv2.imread(image_path, cv2.IMREAD_COLOR)
        if max(image.shape[0], image.shape[1]) > 1024: 
            image = cv2.resize(image, image_size)
        depth_image = depth_map(image)[0]
        cv2.imwrite(dst_name, depth_image)
        if i % 50 == 0:
            print(f'{i+1} / {size}')
    except Exception as e:
        print(f'{i} ({image_path}):', e)
        if not PROD:
            raise e

1 / 178
51 / 178
101 / 178
151 / 178


In [6]:
def read_ldr(path):
    ldr_img = cv2.imread(path, cv2.IMREAD_COLOR)
    ldr_img = cv2.cvtColor(ldr_img, cv2.COLOR_BGR2RGB).astype('float32')
    return (ldr_img / 255).astype('float32')

In [7]:
# image_size = (1024, 1024)
# path = '/home/vovechkin/generator_data/Hday2night/composite_images/d18997-17_1_5.jpg'

# comp = read_ldr(path)
# img = cv2.resize(comp, image_size)